In [20]:
from df_functions import read_csv_file, extract_users, split_user_df, clean_reqlogs
from calculations import longest_consec, calc_avg_timediff, get_variance_score
from sessionID_check import avg_tokens_5mins
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [21]:
"""Code for fetching the data"""

#Fetch data from csv file
data = read_csv_file('requests.csv')

#Extracts a list of all unique users in dataframe
users = extract_users(data)

#Fetches the first user in list
user1 = users[1]

data_chunk = split_user_df(data, user1)[0]

#longest_consec(data_chunk)


In [22]:
df = data.copy()
cleaned = clean_reqlogs(df)
vectorizer = CountVectorizer()
x = vectorizer.fit(cleaned['request_logs'])

df_vectorized = pd.DataFrame(x.todense(), columns= vectorizer.get_feature_names_out())
df_vectorized

AttributeError: 'CountVectorizer' object has no attribute 'todense'

In [19]:
def get_variance_score(dataframe, vectorizer):
    df = dataframe.copy()
    cleaned = clean_reqlogs(df)

    x = vectorizer.transform(cleaned['request_logs'])

    df_vectorized = pd.DataFrame(x.todense(), columns= vectorizer.get_feature_names_out() )
    #return df_vectorized.sort_values(by=["vect_scores"], ascending=False)
    #list = df_vectorized.sort_values(by=["vect_scores"], ascending=False)
    return df_vectorized.var()

get_variance_score(data_chunk, vectorizer)

block          NaN
chat           NaN
config         NaN
conversation   NaN
inbox          NaN
like           NaN
login          NaN
markasread     NaN
page           NaN
photos         NaN
profile        NaN
report         NaN
searchusers    NaN
send           NaN
userconfig     NaN
dtype: float64

In [ ]:
df_vectors = pd.DataFrame()
for user in users:
    data_chunks = split_user_df(data, user)
    avg_req_frequences = []
    avg_tokens = []
    repetitions = []

    for i in range(0, len(data_chunks)):
        avg_req_frequences.append(calc_avg_timediff(data_chunks[i]))
        avg_tokens.append(avg_tokens_5mins(data_chunks[i]))
        repetitions.append(longest_consec(data_chunks[i]))
    
    user_df = pd.DataFrame()
    user_df['request_freq'] = avg_req_frequences
    user_df['avg_tokens'] = avg_tokens
    user_df['longest_consec'] = repetitions
    user_df['user'] = user
    df_calculations = pd.concat([df_calculations, user_df])

df_calculations.reset_index(inplace=True)
df_calculations = df_calculations.drop(columns='index')
df_calculations.to_csv('calculations.csv', index=False) 

In [36]:
"""
Code for creating csv-file containing information about average request frequency, average number of tokens per 5 mins, 
longest consecutive chain of repetitive requests for every chunk of 50 requests for each user
"""

df_calculations = pd.DataFrame()
for user in users:
    data_chunks = split_user_df(data, user)
    avg_req_frequences = []
    avg_tokens = []
    repetitions = []

    for i in range(0, len(data_chunks)):
        avg_req_frequences.append(calc_avg_timediff(data_chunks[i]))
        avg_tokens.append(avg_tokens_5mins(data_chunks[i]))
        repetitions.append(longest_consec(data_chunks[i]))
    
    user_df = pd.DataFrame()
    user_df['request_freq'] = avg_req_frequences
    user_df['avg_tokens'] = avg_tokens
    user_df['longest_consec'] = repetitions
    user_df['user'] = user
    df_calculations = pd.concat([df_calculations, user_df])

df_calculations.reset_index(inplace=True)
df_calculations = df_calculations.drop(columns='index')
df_calculations.to_csv('calculations.csv', index=False)    